In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow import keras;
from tensorflow.keras import layers, metrics;

import tensorflow.keras.metrics as km


In [ ]:
#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SPAM/enron_spam_data.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/trabalho-final-AM/dataset/enron_spam_data.csv')

In [ ]:
df.head()

In [ ]:
df = df.drop_duplicates(keep='first')

In [ ]:
encoder = LabelEncoder()
df['Spam/Ham'] = encoder.fit_transform(df['Spam/Ham'])

In [ ]:
df.head()

In [ ]:
df_train = df.sample(frac=.8, random_state=11)
df_test = df.drop(df_train.index)
print(df_train.shape, df_test.shape)

In [ ]:
df_train = df_train.dropna(subset=['Message'])
df_test = df_test.dropna(subset=['Message'])

In [ ]:
y_train = df_train['Spam/Ham'].values
y_test = df_test['Spam/Ham'].values
y_test.shape

In [ ]:
X_train = df_train['Message'].values
X_test = df_test['Message'].values


In [ ]:
# Converte valores não-string em strings
X_train = [str(text) for text in X_train]
X_test = [str(text) for text in X_test]

## Convertendo texto em sequência númerica

In [ ]:

tokenizer = Tokenizer()
anz_woerter = 500
tokenizer = Tokenizer(num_words=anz_woerter)
tokenizer.fit_on_texts(X_train)
word_dict = tokenizer.index_word

In [ ]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


In [ ]:
laenge_pads = 20

X_train_pad = pad_sequences(X_train_seq, maxlen=laenge_pads, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=laenge_pads)
X_train_pad[:5]
X_train_pad.shape

(26680, 20)

In [ ]:

lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=anz_woerter+1, output_dim=20, input_length=laenge_pads))
lstm_model.add(LSTM(400))
lstm_model.add(Dense(1, activation='sigmoid'))



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
lstm_model.compile(loss='binary_crossentropy', optimizer='Adamax',
                 metrics=['accuracy', km.Precision(), km.Recall()])

lstm_model.build(input_shape=(None, laenge_pads))
lstm_model.summary()


In [ ]:
history = lstm_model.fit(X_train_pad, y_train, epochs=10, batch_size=64,
                        validation_data=(X_test_pad, y_test))

In [ ]:
y_pred_probs = lstm_model.predict(X_test_pad)
# Converte as probabilidades em previsões binárias (0 ou 1)
y_pred = (y_pred_probs > 0.5).astype("int32")


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Calculando as métricas de avaliação
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Acurácia: {accuracy:.4f}')
print(f'Precisão: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

#Visualizar a correlação entre matriz usando a mapa de calor
plt.figure(figsize=(5, 3))
sns.heatmap(confusion_matrix(y_test,y_pred), annot=True, cmap='coolwarm', fmt=".0f", linewidths=.5)
plt.title('Matriz de confusão - LSTM')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.show()

In [ ]:
#https://github.com/burkhardtroeper/RNN-Spam-Detection-LSTM/blob/master/Spam-Detection%20with%20a%20RNN%2C%20using%20Keras-LSTM.ipynb